In [2]:
#XGBoost
import pandas as pd
import numpy as np

In [14]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
%matplotlib inline

In [6]:
from os.path import dirname, join
from pathlib import Path
ROOT_DIR = dirname(os.path.abspath('ARCH7211-SPRING2021')) # This is your Project Root
data_PATH = join(ROOT_DIR,"data","room_info.csv")  # requires `import os`

# input data
data = pd.read_csv(r'C:\Users\milad\Documents\GitHub\arch7211-spring2021\data\room_info.csv')

In [7]:
#Cleaning data and feature engineering
exclusion = ["Undefined","UserDefined","Other"]
merge = {
    "Sauna": "Bath",
    "Den":'LivingRoom',
    "Library": "Office",
    "DressingRoom":'Closet'
}

data["type_cleaned"] = data["type"].apply(lambda x: merge.get(x,x))
data["exclude"] = ~data["type"].isin(exclusion)

In [8]:
data_exclude = data[~data["type"].isin(exclusion)]

In [9]:
data_exclude.groupby("type_cleaned",as_index=False).count().sort_values("type_cleaned")

,type_cleaned,path,type,classes,floor_index,num_sides,area,proportion_floor_area,perimeter,compactness,...,contains_Sink,contains_SpaceForAppliance,contains_Stove,contains_Toilet,contains_TumbleDryer,contains_Urinal,contains_WallCabinet,contains_WashingMachine,contains_WaterTap,exclude
0,Alcove,164,164,164,164,164,164,164,164,164,...,164,164,164,164,164,164,164,164,164,164
1,Attic,93,93,93,93,93,93,93,93,93,...,93,93,93,93,93,93,93,93,93,93
2,Basement,51,51,51,51,51,51,51,51,51,...,51,51,51,51,51,51,51,51,51,51
3,Bath,7288,7288,7288,7288,7288,7288,7288,7288,7288,...,7288,7288,7288,7288,7288,7288,7288,7288,7288,7288
4,Bedroom,7993,7993,7993,7993,7993,7993,7993,7993,7993,...,7993,7993,7993,7993,7993,7993,7993,7993,7993,7993
5,CarPort,279,279,279,279,279,279,279,279,279,...,279,279,279,279,279,279,279,279,279,279
6,Closet,3041,3041,3041,3041,3041,3041,3041,3041,3041,...,3041,3041,3041,3041,3041,3041,3041,3041,3041,3041
7,Dining,954,954,954,954,954,954,954,954,954,...,954,954,954,954,954,954,954,954,954,954
8,DraughtLobby,1667,1667,1667,1667,1667,1667,1667,1667,1667,...,1667,1667,1667,1667,1667,1667,1667,1667,1667,1667
9,Elevated,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100


In [10]:
#Clean input data from the target data
# define features and predictions
X = data_exclude.drop(['path','type','classes','open_to','door_to','contains','type_cleaned','exclude'], axis =1)
y = data_exclude['type']
#Split the data into training data, and test data 
X_train , X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3)

In [32]:
#XGB default parameters
xgb = XGBClassifier(objective='multi:softprob',
                    scale_pos_weight=3,
                    subsample= 0.9,
                    colsample_bytree = 0.6)
print(xgb)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.6, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=3, subsample=0.9,
              tree_method=None, validate_parameters=None, verbosity=None)


In [34]:
xgb.fit(X_train,y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        eval_metric= 'mlogloss',
        early_stopping_rounds=10,
        verbose=True)

[01:30:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-mlogloss:1.23812	validation_1-mlogloss:1.27551
[1]	validation_0-mlogloss:1.01268	validation_1-mlogloss:1.06300
[2]	validation_0-mlogloss:0.86133	validation_1-mlogloss:0.92026
[3]	validation_0-mlogloss:0.75411	validation_1-mlogloss:0.82032
[4]	validation_0-mlogloss:0.66782	validation_1-mlogloss:0.74108
[5]	validation_0-mlogloss:0.60059	validation_1-mlogloss:0.68114
[6]	validation_0-mlogloss:0.54678	validation_1-mlogloss:0.63413
[7]	validation_0-mlogloss:0.50451	validation_1-mlogloss:0.59803
[8]	validation_0-mlogloss:0.46895	validation_1-mlogloss:0.56875
[9]	validation_0-mlogloss:0.43

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=3, subsample=0.9,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [35]:
train_score = xgb.score(X_train,y_train)
print("Training score: ",train_score)

Training score:  0.9561158071451713


In [36]:
# cv_score = cross_val_score(xgb,X_train,y_train, cv=10)
# print("CV mean score: ",cv_score.mean())

In [37]:
best_iteration = xgb.get_booster().best_ntree_limit
y_pred = xgb.predict(X_test, ntree_limit=best_iteration)

In [38]:
print(classification_report(y_test,y_pred))

                precision    recall  f1-score   support

        Alcove       0.78      0.64      0.71        56
         Attic       0.69      0.41      0.51        22
      Basement       0.25      0.06      0.10        16
          Bath       0.96      0.96      0.96      2106
       Bedroom       0.82      0.94      0.88      2412
       CarPort       0.76      0.59      0.67        76
        Closet       0.86      0.93      0.89       800
           Den       0.70      0.45      0.55        99
        Dining       0.91      0.87      0.89       308
  DraughtLobby       0.91      0.92      0.91       494
  DressingRoom       0.68      0.44      0.54       113
      Elevated       0.87      0.43      0.58        30
         Entry       0.89      0.94      0.91      1225
        Garage       0.75      0.75      0.75       126
          Hall       0.74      0.25      0.37        56
       Kitchen       0.96      0.95      0.95      1422
       Library       0.00      0.00      0.00  

In [11]:
#XGB default parameters
xgb = XGBClassifier(objective='multi:softprob',
                    scale_pos_weight=3,
                    subsample= 0.9,
                    colsample_bytree = 0.6)
print(xgb)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.6, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=3, subsample=0.9,
              tree_method=None, validate_parameters=None, verbosity=None)


In [13]:
#Reproductability
seed = 342
np.random.seed(seed)

In [27]:
#Grid Search

#Define CV strategy:
cv = StratifiedKFold(n_splits=5 , shuffle=True, random_state=seed)
#params_grid
params_grid = {
    'max_depth' : [1,3,6],
    'n_estimators' : [200,400],
    'learning_rate' : [0.01, 1] 
                }
params_fixed = {
    'objective' : 'multi:softprob',
                }
bst_grid = GridSearchCV(estimator=XGBClassifier(**params_fixed, seed=seed),param_grid=params_grid, cv = cv, scoring='f1_macro')

In [28]:
fit_params = {
        'eval_set':[(X_train, y_train), (X_test, y_test)],
        'eval_metric':'mlogloss',
        'early_stopping_rounds':10
                }
bst_grid.fit(X_train,y_train,**fit_params)


8617
[97]	validation_0-mlogloss:1.13106	validation_1-mlogloss:1.17896
[98]	validation_0-mlogloss:1.12374	validation_1-mlogloss:1.17189
[99]	validation_0-mlogloss:1.11645	validation_1-mlogloss:1.16487
[100]	validation_0-mlogloss:1.10927	validation_1-mlogloss:1.15796
[101]	validation_0-mlogloss:1.10219	validation_1-mlogloss:1.15114
[102]	validation_0-mlogloss:1.09514	validation_1-mlogloss:1.14438
[103]	validation_0-mlogloss:1.08822	validation_1-mlogloss:1.13772
[104]	validation_0-mlogloss:1.08137	validation_1-mlogloss:1.13116
[105]	validation_0-mlogloss:1.07457	validation_1-mlogloss:1.12461
[106]	validation_0-mlogloss:1.06786	validation_1-mlogloss:1.11816
[107]	validation_0-mlogloss:1.06125	validation_1-mlogloss:1.11182
[108]	validation_0-mlogloss:1.05474	validation_1-mlogloss:1.10555
[109]	validation_0-mlogloss:1.04828	validation_1-mlogloss:1.09934
[110]	validation_0-mlogloss:1.04192	validation_1-mlogloss:1.09324
[111]	validation_0-mlogloss:1.03557	validation_1-mlogloss:1.08713
[112]	va

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=342, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missi...onstraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None,
                                     objective='multi:softprob',
                                     random_state=None, reg_alpha=None,
                                     re

In [41]:
print('Best params:',bst_grid.best_params_)

Best params: {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 400}


In [42]:
bst_pred = bst_grid.predict(X_test)

In [43]:
print(classification_report(y_test,bst_pred))

                precision    recall  f1-score   support

        Alcove       0.70      0.66      0.68        56
         Attic       0.60      0.48      0.53        25
      Basement       0.00      0.00      0.00        16
          Bath       0.97      0.96      0.96      2200
       Bedroom       0.81      0.96      0.88      2377
       CarPort       0.70      0.45      0.55        85
        Closet       0.87      0.92      0.90       828
           Den       0.58      0.47      0.52        81
        Dining       0.88      0.88      0.88       300
  DraughtLobby       0.88      0.93      0.90       501
  DressingRoom       0.62      0.49      0.54       107
      Elevated       0.71      0.18      0.29        28
         Entry       0.91      0.92      0.92      1275
        Garage       0.73      0.70      0.72       134
          Hall       0.88      0.15      0.25        47
       Kitchen       0.97      0.95      0.96      1387
       Library       0.00      0.00      0.00  